In [1]:
import lidar
import rosbag
from pathlib import Path
from dask.distributed import Client
from dask import delayed
import dask.dataframe as dd

client = Client()  # start distributed scheduler locally.  Launch dashboard
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [2]:
bagfile = "/workspaces/lidar/tests/testdata/big.bag"

In [8]:
test = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 100, end_frame_number = 110)

100%|██████████| 10/10 [00:03<00:00,  2.82it/s]


In [7]:
str(test.timestamps[0])

'2020-12-03 10:37:44.596343'

In [9]:
str(test.timestamps[0])

'2020-12-03 10:37:44.596343'

In [15]:
start = 0
stop = 100

chunksize = 10
chunk = 0

for i in range(start,stop):
    print(i)
    if i % chunksize == 0:
        chunk = chunk + 1
        print(f"new chunk {chunk}")


0
new chunk 1
1
2
3
4
5
6
7
8
9
10
new chunk 2
11
12
13
14
15
16
17
18
19
20
new chunk 3
21
22
23
24
25
26
27
28
29
30
new chunk 4
31
32
33
34
35
36
37
38
39
40
new chunk 5
41
42
43
44
45
46
47
48
49
50
new chunk 6
51
52
53
54
55
56
57
58
59
60
new chunk 7
61
62
63
64
65
66
67
68
69
70
new chunk 8
71
72
73
74
75
76
77
78
79
80
new chunk 9
81
82
83
84
85
86
87
88
89
90
new chunk 10
91
92
93
94
95
96
97
98
99


In [8]:
%%time
test = delayed(lidar.Dataset.from_file)(Path(bagfile), start_frame_number = 0, end_frame_number = 100)

CPU times: user 1.28 ms, sys: 1.09 ms, total: 2.37 ms
Wall time: 1.79 ms


In [9]:
%%time
testx = test.compute()

CPU times: user 1.5 s, sys: 1.03 s, total: 2.53 s
Wall time: 34.7 s


In [11]:
testx.to_file(Path("/workspaces/test3"))

Files written to: /workspaces/test3/big


In [5]:
test

Delayed('from_file-a4fbf140-4a9e-45f5-8035-7873bfafb24e')

In [14]:
testx.data.x.max().compute()

AttributeError: 'list' object has no attribute 'x'

In [19]:
data2 = dd.from_delayed(testx.data)

In [21]:
data2.x.max().compute()

1.252312421798706

In [3]:

test0 = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 0, end_frame_number = 100)


100%|██████████| 100/100 [00:47<00:00,  2.09it/s]


In [4]:
test1 = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 101, end_frame_number = 200)


100%|██████████| 99/99 [00:38<00:00,  2.60it/s]


In [5]:
str(test0.start_time)

'2020-12-03 10:37:34.313459'

In [6]:
str(test1.start_time)

'2020-12-03 10:37:44.733687'

In [7]:
newdat = test0.data + test1.data
newtimes = test0.timestamps + test1.timestamps

In [14]:
long=lidar.Dataset(data=newdat,timestamps=newtimes)

In [15]:
long[150]

Frame(               x         y         z  intensity         t  reflectivity  ring  \
0       0.804142 -0.043635  0.238854       57.0         0             4     0   
1       0.810508 -0.013412  0.231400      117.0         0             8     1   
2       0.826164  0.017104  0.227178      113.0         0             8     2   
3       0.808924 -0.013357  0.198374       99.0         0             7     5   
4       0.802965  0.016287  0.188698      147.0         0            10     6   
...          ...       ...       ...        ...       ...           ...   ...   
111918  0.802611  0.045460 -0.145731       76.0  99967600             5    51   
111919  0.805425 -0.043116 -0.153207       54.0  99967600             4    52   
111920  0.783434 -0.013295 -0.156383       95.0  99967600             6    53   
111921  0.808345  0.045884 -0.178086       99.0  99967600             7    55   
111922  0.779879  0.044118 -0.202711       60.0  99967600             4    59   

        noise  range 

In [52]:
long.to_file(Path("/workspaces/test3"))

Files written to: /workspaces/test3/accc6956-af0c-4417-a95f-306bf06586ca


In [60]:
test_in = lidar.Dataset.from_file(Path("/workspaces/test3/accc6956-af0c-4417-a95f-306bf06586ca"))

In [69]:
res = test_in[0].limit("x",-0.1,0)

In [70]:
res.data.x.max()

-1.4917657e-05